In [ ]:
# 모듈 불러오기
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt



In [ ]:
# 금융 데이터 패키지 설치
!pip install finance-datareader

In [ ]:
# 모듈 불러오기
import FinanceDataReader as fdr

In [ ]:
# 포스코(005490) 2000년 ~ 현재일 데이터 가져오기  
posco = fdr.DataReader('005490','2000-01-01','2022-02-18')

In [ ]:
# 데이터 확인
posco.tail() # 마지막 데이터 살펴보기

In [ ]:
posco.shape # 데이터 크기

In [ ]:
posco.info() # 데이터 유형 

In [ ]:
plt.figure(figsize=(16, 9))    # plt 사이즈 설정
posco["Close"].plot()          # 주가 플랏 그리기

In [ ]:
# 정규화, 모든 값을 0과 1사이로 리스케일링, 극단값에 의한 영향을 최소화
scaler = MinMaxScaler()
scale_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
df_scaled = scaler.fit_transform(posco[scale_cols])

df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols
df_scaled.head()


In [ ]:
df_scaled.shape

In [ ]:
# 데이터셋 분할
train = df_scaled[:5000]
test = df_scaled[5000:]
train.shape, test.shape

In [ ]:
# 20일 단위로 데이터를 묶어주는 함수
def make_dataset(data, label, window_size):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [ ]:
# feature 와 label 정의
feature_cols = ['Open', 'High', 'Low', 'Volume']
label_cols = ['Close']
train_feature = train[feature_cols]
train_label = train[label_cols]
train_feature, train_label = make_dataset(train_feature, train_label, 20)



In [ ]:
train_feature.shape, train_label.shape   

In [ ]:
train_feature[0:2]

In [ ]:
# train, validation set 생성, 학습시 20%는 검증용으로 활용
x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)
x_train.shape, x_valid.shape  

In [ ]:
# test 데이터도 20일 단위로 묶어 주기 (실제 예측할 데이터)
test_feature = test[feature_cols]
test_label = test[label_cols]
test_feature, test_label = make_dataset(test_feature, test_label, 20)
test_feature.shape, test_label.shape

In [ ]:
# 모델 정의
model = tf.keras.models.Sequential([
  tf.keras.layers.LSTM(input_shape=(20, 4), units=16, activation='tanh'),
  tf.keras.layers.Dense(1, activation='sigmoid')])

In [ ]:
model.summary()

In [ ]:
# 학습하기
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
history = model.fit(x_train, y_train, 
                    epochs=200, 
                    batch_size=16,
                    validation_data=(x_valid, y_valid))


In [ ]:
# 예측하기
pred = model.predict(test_feature)

In [ ]:
# 테스트 데이터와 예측 데이터 비교 플랏
plt.figure(figsize=(12, 9))
plt.plot(test_label, label='actual')
plt.plot(pred, label='prediction')
plt.legend()
plt.show()